In [1]:
import pandas as pd

In [2]:
file_path = "shopping_data.csv"
df_shopping = pd.read_csv(file_path, encoding="ISO-8859-1")
df_shopping.head(5)

,CustomerID,Card Member,Age,Annual Income,Spending Score (1-100)
0,1,Yes,19.0,15000,39.0
1,2,Yes,21.0,15000,81.0
2,3,No,20.0,16000,6.0
3,4,No,23.0,16000,77.0
4,5,No,31.0,17000,40.0


First, account for the data you have. After all, you can't extract knowledge without data. We can use the columns method and output the columns, as shown below:

In [5]:
# Columns
df_shopping.columns

Index(['CustomerID', 'Card Member', 'Age', 'Annual Income',
       'Spending Score (1-100)'],
      dtype='object')

Using the dtypes method, confirm the data type, which also will alert us if anything should be changed in the next step (e.g., converting text to numerical data). All the columns we plan to use in our model must contain a numerical data type:

In [4]:
#List dataframe data types
# Columns
df_shopping.dtypes

CustomerID                  int64
Card Member                object
Age                       float64
Annual Income               int64
Spending Score (1-100)    float64
dtype: object

Next, let's see if any data is missing. Unsupervised learning models can't handle missing data. If you try to run a model on a dataset with missing data, you'll get an error such as the one below:

If you initially had hoped to produce an outcome using a type of data, but it turned out more than 80% of those rows are empty, then the results won't be very accurate!

Pandas has the isnull() method to check for missing values. We'll loop through each column, check if there are null values, sum them up, and print out a readable total:

In [6]:
#Find null values 
for column in df_shopping.columns:
    print(f'Column {column} has {df_shopping[column].isnull().sum()} null values')

Column CustomerID has 0 null values
Column Card Member has 2 null values
Column Age has 2 null values
Column Annual Income has 0 null values
Column Spending Score (1-100) has 1 null values


There will be a few rows with missing values that we'll need to handle. The judgement call will be to either remove these rows or decide that the dataset is not suitable for our model. In this case, we'll proceed with handling these values because they are a small percentage of the overall data.

What data can be removed?
You have begun to explore the data and have taken a look at null values. Next, determine if the data can be removed. Consider: Are there string columns that we can't use? Are there columns with excessive null data points? Was our decision to handle missing values to just remove them?

In our example, there are no string type columns, and we made the decision that only a few rows have null data points, but not enough to remove a whole column's worth.

Rows of data with null values can be removed with the dropna() method, as shown below:

In [7]:
# Drop null rows
df_shopping = df_shopping.dropna()

Duplicates can also be removed.Use the duplicated().sum() method to check for duplicates, as shown below:

In [8]:
#Find duplicate entries 
print(f'Duplicate entries:{df_shopping.duplicated().sum()}')

Duplicate entries:0


In [9]:
# Remove the CustomerID Column
df_shopping.drop(columns=["CustomerID"],inplace=True)
df_shopping.head()

,Card Member,Age,Annual Income,Spending Score (1-100)
0,Yes,19.0,15000,39.0
1,Yes,21.0,15000,81.0
2,No,20.0,16000,6.0
3,No,23.0,16000,77.0
4,No,31.0,17000,40.0


For data processing, the focus is on making sure the data is set up for the unsupervised learning model, which requires the following:

Null values are handled. Only numerical data is used. Values are scaled. In other words, data has been manipulated to ensure that the variance between the numbers won't skew results.

Recall that when features have different scales, they can have a disproportionate impact on the model. The unscaled value could lead to messy graphs. Therefore, it is important to understand when to scale and normalize data. For example, if four columns of data are single digits, and the fifth column is in the millions, we would need to scale the fifth column to align the other four.

Is the data in a format that can be passed into an unsupervised learning model?
We saw before that all our data had the correct type for each column; however, we know that our model can't have strings passed into it.

To make sure we can use our string data, we'll transform our strings of Yes and No from the Card Member column to 1 and 0, respectively, by creating a function that will convert Yes to a 1 and anything else to 0.

The function will then be run on the whole column with the .apply method, as shown below:

In [10]:
#Transform string column 
def change_string(member):
    if member == "Yes":
        return 1 
    else:
        return 0
df_shopping["Card Member"] = df_shopping["Card Member"].apply(change_string)
df_shopping.head()

,Card Member,Age,Annual Income,Spending Score (1-100)
0,1,19.0,15000,39.0
1,1,21.0,15000,81.0
2,0,20.0,16000,6.0
3,0,23.0,16000,77.0
4,0,31.0,17000,40.0


Also, there is one more thing you may notice about the data. The scale for Annual Income is much larger than all the other values in the dataset. We can adjust this format by dividing by 1,000 to rescale those data points, as shown below:

In [11]:
# Transform annual income 
df_shopping["Annual Income"] = df_shopping["Annual Income"] / 1000
df_shopping.head()

,Card Member,Age,Annual Income,Spending Score (1-100)
0,1,19.0,15.0,39.0
1,1,21.0,15.0,81.0
2,0,20.0,16.0,6.0
3,0,23.0,16.0,77.0
4,0,31.0,17.0,40.0


In [12]:
#Reformat the names of the columns
df_shopping["Spending Score"] = df_shopping["Spending Score (1-100)"]
df_shopping.drop(columns=["Spending Score (1-100)"],inplace=True)
df_shopping.head()

,Card Member,Age,Annual Income,Spending Score
0,1,19.0,15.0,39.0
1,1,21.0,15.0,81.0
2,0,20.0,16.0,6.0
3,0,23.0,16.0,77.0
4,0,31.0,17.0,40.0


Data transformation involves thinking about the future. More times than not, there will be new data coming into your data storage (a place where raw data is stored before being touched), with many people working on different types of data analysis. We want to make sure that whoever wants to use the data in the future can do so.

Now that our data has been cleaned and processed, it is ready to be converted to a readable format for future use:

In [13]:
# Saving cleaned data
file_path = "shopping_data_cleaned.csv"
df_shopping.to_csv(file_path, index=False)
